## Importo librerias

In [1]:
# Librerías principales
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configuración general
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', None)


## Localización Proyecto y carga de Dataset

In [2]:
# Detectar la raíz real del proyecto (carpeta que contiene "DATA")
PROJECT_ROOT = Path.cwd().resolve()

# Si estás dentro de NOTEBOOKS, sube un nivel
if PROJECT_ROOT.name.lower() == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent

# Si aún no ves DATA, sube hasta encontrarla
while not (PROJECT_ROOT / "DATA").exists() and PROJECT_ROOT.parent != PROJECT_ROOT:
    PROJECT_ROOT = PROJECT_ROOT.parent

# Verificación de seguridad
if not (PROJECT_ROOT / "DATA").exists():
    raise FileNotFoundError("No encuentro la carpeta 'DATA'. Abre el notebook dentro del proyecto.")





# Carga
FNAME = "dataset_limpio_y_transformado.csv"
ruta_dataset = PROJECT_ROOT / "DATA" / "DATA_OUTPUT" / FNAME

# Comprobación y carga segura
if not ruta_dataset.exists():
    # Búsqueda recursiva en caso de estar en otra subcarpeta
    hits = list(PROJECT_ROOT.rglob(FNAME))
    if hits:
        ruta_dataset = hits[0]

if not ruta_dataset.exists():
    raise FileNotFoundError(
        f"No encuentro el archivo {FNAME}. Vuelve a exportarlo a DATA/DATA_OUTPUT/ o revisa su nombre.\n"
        f"Probé en: {ruta_dataset.parent}"
    )




# 💳Informe del Análisis Exploratorio de Datos sobre operaciones financieras etiquetadas como fraude / no fraude
### 🧠 Proyecto Final – Máster en Data Analytics
---
---


## 1. Introducción

### 🎯 Objetivo del análisis
Realizar un **análisis exploratorio de datos (EDA)** sobre un conjunto de **operaciones financieras** para:
- **Identificar patrones, relaciones y tendencias** asociados a **fraude vs. no fraude**.
- Entender cómo variables como **comercio, tipo de comercio, nivel de riesgo (alto/medio/bajo), antigüedad, tipo de operación, tipo de tarjeta, resultado de la operación** se relacionan con la probabilidad de fraude.
- Generar **insights accionables** que ayuden a **priorizar revisiones**, **afinar reglas** y **mejorar la prevención** y la **gestión operativa del fraude**.

Preguntas guía:
- ¿Cuál es la **prevalencia de fraude** global y por **segmentos** (merchant / tipo de comercio / nivel de riesgo / tipo de operación)?
- ¿Existen **picos temporales** de fraude (por día/mes/hora o por antigüedad de la cuenta)?
- ¿Qué **combinaciones de variables** incrementan el riesgo.
- ¿Qué diferencias hay por variables **geográficas** (*recordatorio: `region` = región del cliente; `country` = país donde se realiza la operación*)?
- ¿Qué **insights** pueden convertirse en **acciones** concretas?

Estas preguntas actúan como guía del análisis y orientan la búsqueda de patrones en los datos.

Cada una de ellas se abordará a lo largo del informe mediante gráficos, métricas y conclusiones .



---

### 🧩 Contexto del dataset

El análisis parte de **dos archivos originales**:

1) **Clientes (original)** → contiene información básica del cliente: customer_id, nombre, email, phone, region, country, created_at, risk_core.  
2) **Transacciones (original)** → recoge el detalle de operación: transaction_id, customer_id, transaction_time, amount, currency, merchant, merchant_category, arn, entry_mode, card_bin6, card_last4, card_masked, card_tipe, card_expiry, transation_result, is_fraud. 

Ambos archivos se **unieron mediante una columna en común** (identificador de cliente o cuenta), dando lugar a un **único dataset consolidado** que integra tanto el perfil del cliente como la información transaccional.  

El resultado de esta unión se sometió posteriormente a un proceso de **limpieza y transformación**, del cual surgió el archivo final **`dataset_limpio_y_transformado.csv`**.  
Este último representa la **fuente única de verdad** del proyecto y es el que se utiliza en el análisis exploratorio y en todo el informe.


**Alcance:** Este informe es **descriptivo** (EDA). No incluye modelos predictivos; se centra en **explorar** y **explicar** el comportamiento del fraude con visualizaciones y métricas descriptivas.


---

### 📘 Estructura del informe
1. **Introducción** — Objetivos, contexto y alcance.  
2. **Datos y calidad** — Estructura del dataset final y evaluación de calidad.  
3. **Principales transformaciones** — Resumen de limpieza, uniones y derivadas clave.  
4. **Análisis visual (6–8 gráficos)** — Distribuciones, comparativas y relaciones con fraude.  
5. **Insights y recomendaciones** — *Usar siempre “insights” (no “hallazgos”)* y traducirlos a acciones.  
6. **Limitaciones del estudio** — Restricciones y consideraciones del dato.  
7. **Próximos pasos** — Profundizaciones, datos adicionales y líneas de mejora.

---
---


## 📈 2. Datos y calidad

### 2.1. Origen y consolidación
El análisis se ha desarrollado a partir de un conjunto de **datos sintéticos** que simulan operaciones financieras reales.  
El proyecto parte de **dos fuentes iniciales**:

1. **Clientes** → información general del cliente (identificador, correo, teléfono, país, etc.).  
2. **Transacciones** → operaciones financieras con detalle de comercio, importe, tipo de tarjeta, modo de entrada, nivel de riesgo y etiqueta de fraude.

Ambos conjuntos se **unieron mediante una columna en común (`customer_id`)**, generando un **dataset consolidado** que integra el comportamiento transaccional con los atributos del cliente.  
Posteriormente, se llevó a cabo un proceso de **limpieza, transformación y normalización**, cuyo resultado final se guardó en el archivo:

> **`dataset_limpio_y_transformado.csv`**  
> Este archivo constituye la **fuente única de verdad** sobre la que se ha desarrollado todo el análisis exploratorio.


---

### 2.2. Estructura y volumen
- **Filas:** **39.906**  
- **Columnas:** **34**  
- **Tasa global de fraude (`is_fraud`):** **1,10 %**

Variables relevantes:

- `is_fraud`  → indicador principal (1 = fraude, 0 = no fraude). 
- `risk_level`  → clasificación de riesgo (alto, medio, bajo).
- `entry_mode` → modo de entrada de la transacción (*chip, contactless, magstripe, wallet, ecommerce*).   
- `transaction_result` → resultado de la operación (*approved, declined, pending*). 
- `card_type`  → tipo de tarjeta (*VISA, MASTERCARD, AMEX, DISCOVER*).
- `merchant_category`   → categoría o tipo de comercio (*electronics, fashion, grocery, travel, etc.*).
- `amount`   → importe de la transacción.  
- `risk_score`   → puntuación numérica de riesgo asignada a la transacción.
- `region_normalized`   → región de residencia o país del cliente. 
- `country_normalized`  → país donde se ha realizado la operación.
- `customer_days_active` → número de días que el cliente llevaba activo en el sistema.
---

### 2.3. Completitud y nulos
- **Nulos totales en el dataset:** **0**.  
- **Nulos por columna:** **ninguna columna** presenta valores nulos tras la consolidación.

---

### 2.4. Integridad y duplicados
- **Duplicados en `transaction_id`:** **0** → cada transacción es única.  
- **Relación cliente–transacción:** consistente **1–a–N** (sin pérdidas evidentes en la clave).

---

### 2.5. Tipos y rangos (checks básicos)
- **Importe (`amount`):**  
  - **mín:** 2,35 · **q25:** 111,76 · **mediana:** 187,69 · **q75:** 261,12 · **máx:** 656,13  
  - **Importes negativos:** **0**
- **Divisa (`currency`):** **EUR** (única).
- **Categorías principales:**
  - `risk_level`: **Alto, Medio, Bajo**  
  - `entry_mode`: **chip, contactless, ecommerce, magstripe, wallet**  
  - `transaction_result`: **approved, declined, pending**  
  - `card_type`: **AMEX, DISCOVER, MASTERCARD, VISA**  
  - `merchant_category`: **electronics, fashion, gaming, grocery, pharmacy, restaurant, travel**

---

### 2.6. Fechas y coherencia temporal
- **`transaction_date`**: de **2024-10-11** a **2025-10-12**  
- **`created_at`**: de **2024-06-17** a **2026-02-02**

**Chequeo de coherencia (`transaction_date` vs `created_at`):**  
- Registros con **transacción anterior a `created_at`**: **19.899**.

> **Interpretación:** este comportamiento sugiere que **`created_at` no representa necesariamente la “fecha de alta inicial”** de la cuenta/tarjeta, sino **otro hito (p. ej., fecha de alta del perfil en sistema actual, migración o actualización)**.  
> Por ello, para medir antigüedad operativa se utiliza **`customer_days_active`**, no la comparación directa `transaction_date < created_at`.

---

### 2.7. Normalización y geografía
- **País/Región:**  
  - `region_normalized` (residencia del cliente) y `country_normalized` (país de operación) están **normalizados** (nombres consistentes).  
  - Existe bandera **`is_international`** para detectar operaciones fuera del país del cliente.  
- **Codificaciones en minúsculas/estandarizadas** en: `entry_mode`, `transaction_result`, `merchant_category`.  
  

---

### 2.8. Conclusión 
El dataset final presenta **excelente completitud (0 nulos)**, **unicidad por transacción**, valores monetarios **dentro de rangos plausibles** y categorías **bien estandarizadas**.  
La única consideración de calidad es la **interpretación de `created_at`**, que no debe usarse como “fecha de alta original” al comparar con `transaction_date`; para antigüedad se dispone de **`customer_days_active`**.  
Con estas garantías, el conjunto es adecuado para analizar el **comportamiento del fraude** por riesgo, tarjeta, comercio, geografía y resultado de la operación.

---
---

## 🔄 3. Principales transformaciones

### 3.1. Consolidación de fuentes
- **Unión** de los dos orígenes (clientes + transacciones) mediante **`customer_id`**.
- Se conservó **toda la casuística transaccional** (join 1–N desde transacciones), generando un único dataset consolidado.
- Resultado de la consolidación: **`dataset_limpio_y_transformado.csv`** (fuente única del EDA).


---

### 3.2. Conversión de tipos
- **Fechas** a `datetime`: `transaction_date`, `created_at`.
- **Numéricos**: `amount` y `risk_score` a tipo numérico.
- **Categóricas**: `risk_level`, `entry_mode`, `transaction_result`, `card_type`, `merchant_category`, `region_normalized`, `country_normalized`.

---

### 3.3. Normalización y estandarización de categorías
- **Etiquetas consistentes** en:
  - `risk_level` → **Alto / Medio / Bajo**  
  - `entry_mode` → **chip / contactless / ecommerce / magstripe / wallet**  
  - `transaction_result` → **approved / declined / pending**  
  - `card_type` → **AMEX / DISCOVER / MASTERCARD / VISA**
- **Geografía normalizada**:
  - `region_normalized` = país/región del cliente (normalizado).
  - `country_normalized` = país de la operación (normalizado).
  - **`is_international`** = indicador si `country_normalized` ≠ `region_normalized`.

---

### 3.4. Variables temporales derivadas
- Desde `transaction_date` se generaron:
  - **`month`** (1–12), **`weekday`** (0–6), **`hour`/`transaction_hour`**, **`month_year`** (etiqueta Y–M).
- Objetivo: habilitar **series temporales**, estacionalidad y análisis por franjas horarias.

---

### 3.5. Antigüedad operativa del cliente
- **`customer_days_active`**: días de actividad del cliente disponibles en el dataset y utilizados para los análisis por madurez.
- **Nota técnica:** 
Se intentó calcular la antigüedad restando `transaction_date - created_at`, pero se detectaron valores **negativos** en una parte significativa de los registros.
Esto ocurre porque `created_at` no representa la fecha de alta original, sino una fecha más reciente (posiblemente de migración o actualización de sistema).
Por ello, se decidió **no utilizar esa resta directa** y en su lugar emplear la variable `customer_days_active`, que refleja correctamente los días de actividad acumulados del cliente.

---

### 3.6. Campos de tarjeta y comercio
- **Tarjeta**: `card_type` (marca), `card_bin6`, `card_last4`, `card_masked`, `card_expiry` (soporte analítico/operativo).
- **Comercio**: `merchant` y **`merchant_category`** (electronics, fashion, gaming, grocery, pharmacy, restaurant, travel).

---

### 3.7. Controles de calidad posteriores a las transformaciones
 Se identificaron **filas incompletas** en las que únicamente figuraba el campo `TransactionID`, mientras que el resto de columnas se encontraban vacías. Estas filas se eliminaron al no aportar información analítica.   
- **Nulos**: **0** nulos en el dataset (todas las columnas).
- **Duplicados** en `transaction_id`: **0**.
- **Importes (`amount`)**: rango **[2.35 ; 656.13]**, sin valores negativos.
- **Fechas**: `transaction_date` en **[2024-10-11 ; 2025-10-12]**; `created_at` en **[2024-06-17 ; 2026-02-02]**.  
  Se documenta la **posible desalineación semántica** de `created_at` (no usar como “alta original”).

---

### 3.8. Filtros y decisiones de conservación
- Se conservaron **todas las categorías** de `entry_mode`, `transaction_result`, `card_type` y `merchant_category` para mantener representatividad.

---
### 3.9. Agrupaciones clave empleadas en el EDA (sin modificar el dataset)

En este notebook se desarrollaron distintos **análisis cruzados y resúmenes estadísticos** orientados a identificar patrones de fraude en la base de datos.  
Cada cruce se diseñó con una finalidad analítica específica, priorizando la relación entre variables clave como `is_fraud`, `merchant_category`, `entry_mode`, `risk_level`, y `risk_score`.

---

#### Tablas y cruces analíticos realizados


| Tabla / Variable derivada | Descripción | Métricas calculadas | Enfoque |
|-----------------------------|--------------|----------------------|----------|
| `res_merchant` | Agrupación por **categoría de comercio (`merchant_category`)**. | Número total de operaciones, fraudes, tasa de fraude, importe medio/mediano, riesgo medio, peso de operaciones y fraudes. | Identificar los **sectores con mayor exposición al fraude**. |
| `res_merchant_by_type` | Agrupación doble por **tipo de comercio y tipo de transacción (fraude/no fraude)**. | Importe medio, importe mediano, riesgo medio. | Comparar el **comportamiento económico de fraudes vs no fraudes** por sector. |
| `top_by_rate` | Ranking de las categorías con **mayor tasa de fraude**. | Tasa de fraude ordenada descendente. | Determinar los **comercios prioritarios en vigilancia antifraude**. |
| `plot_df` (comparativa fraude vs no fraude) | Filtrado de los 10 principales comercios por volumen. | Importe medio agrupado por tipo de transacción. | Analizar la **variación de importes medios** entre fraudes y operaciones legítimas. |
| `fraude_por_entry` | Agrupación por **modo de entrada (`entry_mode`)**. | Porcentaje de fraude por tipo de operación. | Evaluar **qué canales presentan mayor incidencia de fraude**. |
| `freq_entry` | Frecuencia total de operaciones por modo de entrada. | Porcentaje de uso. | Relacionar el **volumen operativo con la tasa de fraude**. |
| `analisis_entry` | Unión de `fraude_por_entry` y `freq_entry`. | Tasa de fraude (%) y frecuencia de uso (%). | Permite visualizar la **relación entre popularidad del canal y su vulnerabilidad**. |
| `tabla_entry_risk` | Cruce entre **entry_mode y risk_level** mediante tabla cruzada. | Distribución porcentual de niveles de riesgo (Alto, Medio, Bajo) por canal. | Determinar **qué modos de entrada concentran mayor proporción de riesgo alto o medio**. |
| `resumen_entry` | Síntesis final de prevalencia y volumen por `entry_mode`. | Número de operaciones y tasa de fraude. | Combinar **exposición (volumen)** y **prevalencia (fraude)** en una misma visión global. |

---

#### Objetivo de las agrupaciones y cruces

El propósito de las agrupaciones desarrolladas en este notebook es ofrecer una **visión analítica integral del fraude**, combinando indicadores de frecuencia, valor económico y riesgo operativo.

#### 🎯 Objetivos específicos:

1. **Detectar patrones de fraude por tipo de comercio**  
   - Identificar los sectores con **mayor tasa y volumen de fraudes**.  
   - Determinar si los fraudes se concentran en operaciones de alto importe o riesgo.  

2. **Evaluar la influencia del canal de entrada (`entry_mode`) en la probabilidad de fraude**  
   - Analizar qué métodos (wallet, magstripe, chip, etc.) presentan **mayor vulnerabilidad**.  
   - Relacionar el uso masivo de ciertos canales con la efectividad de los controles.  

3. **Examinar la relación entre riesgo y fraude**  
   - A través de la tabla `entry_mode` vs `risk_level`, se identifican los **modos con mayor proporción de riesgo medio-alto**, reforzando el análisis predictivo.  

4. **Relacionar volumen de operaciones y prevalencia de fraude**  
   - Mediante resúmenes combinados, se detectan canales con **alta actividad operativa y elevada incidencia de fraude**, priorizando la mitigación en esos puntos.  

5. **Visualizar hallazgos mediante gráficos y rankings claros**  
   - Las visualizaciones (heatmaps, barras comparativas y rankings) aportan una **visión inmediata de las áreas críticas**, favoreciendo la toma de decisiones basada en datos.


Estos análisis constituyen la base de los **gráficos e insights** que se desarrollan posteriormente en el informe.

**Resultado:** tras estas transformaciones, el dataset quedó **coherente, completo (0 nulos)** y con variables **normalizadas y derivadas** que permiten explicar el fraude por **canal, riesgo, tarjeta, comercio, geografía, temporalidad y resultado** sin pérdida de información.

---
---


## 🧩 Archivos generados en la carpeta EDA

Durante el desarrollo del Notebook 03 se generaron múltiples archivos en formato `.csv` y `.png` dentro del directorio **`DATA/DATA_OUTPUT/EDA`**.  
Estos archivos constituyen la **base de visualización y análisis complementario** del informe de fraude y se integrarán posteriormente en el dashboard final de **Power BI**.

A continuación se detallan los archivos, su función analítica y su uso previsto:

---

### 📊 1️⃣ Tablas y datasets exportados (`.csv`)

| Archivo | Descripción | Uso posterior |
|----------|--------------|---------------|
| **amount_por_risk_level.csv** | Tabla resumen del importe medio y total por nivel de riesgo (`risk_level`). | Servirá para construir gráficos de **distribución económica por nivel de riesgo** en Power BI. |
| **comparativa_fraude.csv** | Comparación de métricas clave entre operaciones fraudulentas y no fraudulentas. | Base para visualizaciones comparativas de **fraude vs no fraude** (barras agrupadas o indicadores KPI). |
| **distribucion_fraude.csv** | Resumen global de distribución de fraudes por distintas dimensiones (tipo, fecha, canal). | Referencia para gráficos de **evolución temporal y proporciones globales de fraude**. |
| **resumen_estadistico.csv** | Consolidado de todas las métricas descriptivas por categoría de comercio (`merchant_category`). | Fuente principal para el **resumen estadístico de comercios** y tablas dinámicas en Power BI. |

---

### 📈 2️⃣ Visualizaciones exportadas (`.png`)

#### 🔸 Distribución temporal y operativa
| Archivo | Descripción | Uso previsto |
|----------|--------------|---------------|
| **Distribucion_fraudes_por_mes.png** | Evolución mensual del número de fraudes detectados. | Gráfico de tendencia en Power BI para analizar **patrones estacionales**. |
| **Operaciones_fraudulentas_dia.png** | Distribución diaria de fraudes dentro del mes o periodo. | Visualización de **picos de fraude por día**. |
| **Operaciones_fraudulentas_hora_dia.png** | Análisis de fraudes por hora del día. | Identificar **franjas horarias críticas**. |
| **Operaciones_fraudulentas_semana.png** | Frecuencia de fraudes por día de la semana. | Estudiar **comportamientos recurrentes semanales**. |

---

#### 🔸 Análisis modo de entrada
| Archivo | Descripción | Uso previsto |
|----------|--------------|---------------|
| **Distribucion_general_entry_mode.png** | Reparto general de operaciones por tipo de entrada (`entry_mode`). | Comparación de **frecuencia modo de entrada de operación**. |
| **entrymode_risklevel.png** | Mapa de calor con el % de riesgo por tipo de entrada. | Evaluar **modo de entrada con mayor nivel de riesgo medio/alto**. |
| **fraude_por_resultado_transaccion.png** | Análisis de resultados de transacción y su relación con el fraude. | Ilustrar **efectividad o fallos de autorización**. |

---

#### 🔸 Análisis por tipo de comercio y producto
| Archivo | Descripción | Uso previsto |
|----------|--------------|---------------|
| **importe_medio_por_categoria_comercio.png** | Importe medio de operaciones por categoría de comercio. | Visualización económica general. |
| **tasa_fraude_por_categoria_comercio.png** | Porcentaje de fraude por categoría. | Gráfico de **ranking de sectores más afectados**. |
| **tasa_fraude_topN_merchant.png** | Top N categorías con mayor tasa de fraude. | Gráfico destacado de **comercios de alto riesgo**. |
| **volumen_por_categoria_comercio.png** | Volumen total de operaciones por categoría. | Complemento para el análisis de **peso operativo**. |

---

#### 🔸 Comparativas de fraude
| Archivo | Descripción | Uso previsto |
|----------|--------------|---------------|
| **importe_medio_fraude_vs_no_fraude_top10.png** | Comparación del importe medio entre fraudes y no fraudes (Top 10 comercios). | **Barras agrupadas** para diferenciar comportamiento monetario entre ambos tipos. |
| **comparativa_fraude.csv (relacionado)** | Datos base para este gráfico. | Fuente tabular de Power BI. |

---

#### 🔸 Variables de riesgo y geolocalización
| Archivo | Descripción | Uso previsto |
|----------|--------------|---------------|
| **geo_risk_score_violin.png** | Distribución del `risk_score` por región o país (violin plot). | Análisis geográfico y de dispersión de riesgo. |
| **amount_por_risk_level.png** | Importe medio por nivel de riesgo. | Visualización de **impacto económico por nivel de riesgo**. |

---

#### 🔸 Tipología y resultado de tarjetas
| Archivo | Descripción | Uso previsto |
|----------|--------------|---------------|
| **fraude_por_tipo_tarjeta.png** | Tasa de fraude según la marca o red de la tarjeta. | Comparación de **esquemas de pago con mayor incidencia de fraude**. |
| **operaciones_tipo_tarjeta_comercio.png** | Cruce entre tipo de tarjeta y categoría de comercio. | **Análisis multidimensional**: tipo de tarjeta + sector afectado. |

---

#### 🔸 Análisis global por país
| Archivo | Descripción | Uso previsto |
|----------|--------------|---------------|
| **top_paises_tasa_fraude.png** | Ranking de países con mayor tasa de fraude. | Visualización **geográfica en Power BI** mediante mapa temático. |

---

### 🧭 Conclusión

Estos archivos conforman el **repositorio base de evidencias visuales y tabulares** del análisis exploratorio de fraude.  
Serán utilizados como insumos en el **informe de visualización final** y en el **dashboard de Power BI**, permitiendo:

- Centralizar resultados del análisis descriptivo.  
- Comparar indicadores entre países, comercios, y modos de entrada.  
- Visualizar patrones de riesgo y evolución temporal del fraude.  

📌 En resumen, esta carpeta **EDA** contiene todos los resultados validados y exportados del análisis, garantizando una trazabilidad completa entre el *Notebook 03* y las visualizaciones dinámicas del informe final.

---
---

# 👁️‍🗨️ 4. Análisis visual 

A continuación se presentan las principales visualizaciones del análisis exploratorio, centradas en la detección de patrones de fraude según canal, tipo de tarjeta, riesgo, comercio, resultado y evolución temporal.  
Cada gráfico incluye una breve interpretación (“insight”) basada en los datos del EDA.

---

## 📊 1. Relación entre frecuencia de uso y tasa de fraude según el modo de entrada (`entry_mode`)
![Distribución por entry_mode](../DATA/DATA_OUTPUT/EDA/Distribucion_general_entry_mode.png)



El gráfico muestra la **relación entre el porcentaje total de operaciones** y la **tasa de fraude** para cada tipo de modo de entrada (`entry_mode`), representando así tanto la **popularidad de uso** como el **riesgo relativo** asociado a cada canal de transacción.



Esta visualización permite identificar **qué modo de entrada concentra mayor volumen de uso** y **en cuáles se presenta una mayor proporción de fraude**.

#### 📈 Insights:
1. **`wallet` (billeteras digitales)**  
   - Es el canal con **mayor frecuencia de uso (≈20%)** y también muestra la **tasa de fraude más elevada (≈1.28%)**.  
   - Indica que, aunque es el medio más utilizado, **representa un foco importante de riesgo** y requiere un seguimiento reforzado.  

2. **`magstripe` y `contactless`**  
   - Mantienen una participación similar (≈19–20% de operaciones), con tasas de fraude algo menores (≈1.1–1.15%).  
   - Se consideran **canales moderadamente seguros**, aunque su alto volumen los convierte en **zonas potenciales de exposición**.  

3. **`ecommerce`**  
   - A pesar de su **volumen de uso cercano al 20%**, presenta una **tasa de fraude inferior (≈0.98%)**.  
   - Esto sugiere que **las medidas antifraude implementadas en pagos online están resultando efectivas**, o bien que las transacciones sospechosas son detectadas antes de completarse.  

4. **`chip` (EMV)**  
   - Es el canal **más seguro**, con la **tasa de fraude más baja (≈0.96%)**.  
   - Este resultado es consistente con la tecnología de chip, que incorpora **mecanismos criptográficos más robustos** frente a clonaciones y usos indebidos.

#### 🧩 Conclusiones
- Existe una **tendencia descendente clara**: a mayor seguridad tecnológica del método de entrada, menor es la tasa de fraude.  
- Sin embargo, los canales **más cómodos o rápidos para el usuario** (wallet y contactless) tienden a ser **más susceptibles a intentos de fraude**.  
- Estos resultados pueden utilizarse para **priorizar controles o campañas de prevención** según el canal de entrada más expuesto.  


---

## 💳 2. Porcentaje de fraude por tipo de tarjeta (`card_type`)
![Fraude por tipo de tarjeta](../DATA/DATA_OUTPUT/EDA/fraude_por_tipo_tarjeta.png)





El gráfico muestra la **tasa de operaciones fraudulentas (%)** según el tipo de tarjeta utilizada, permitiendo identificar **qué marcas presentan mayor vulnerabilidad** frente al fraude.



#### 📈 Insights:
1. **AMEX**  
   - Registra la **tasa de fraude más alta (≈1.38%)**, superando ligeramente al resto.  
   - Este resultado podría estar asociado a su **perfil de cliente más internacional** y al uso frecuente en **entornos digitales o de alto importe**, donde la exposición al fraude suele ser mayor.

2. **MASTERCARD**  
   - Presenta una tasa de fraude similar (≈1.32%).  
   - Aunque ampliamente distribuida, su uso masivo en distintos comercios la convierte en un **objetivo habitual de intentos de fraude**.

3. **DISCOVER**  
   - Muestra una tasa intermedia (≈1.18%), manteniendo un comportamiento relativamente estable.  
   - Indica una **exposición moderada**, posiblemente por su menor volumen de operaciones globales.

4. **VISA**  
   - Es la tarjeta **con menor tasa de fraude (≈0.90%)**, lo que sugiere **mayor efectividad de sus controles antifraude y autenticación**.  
   - Puede relacionarse con una **amplia red de detección temprana** y adopción de estándares EMV en la mayoría de sus operaciones.

#### 🧩 Conclusiones
- Se observa una **brecha de riesgo de aproximadamente 0.5 puntos porcentuales** entre la marca más vulnerable (AMEX) y la más segura (VISA).  
- Las diferencias pueden deberse tanto a **estrategias de control de riesgo propias de cada red** como a la **naturaleza de los clientes y comercios asociados**.  
- Este análisis permite **priorizar la vigilancia y los controles adicionales** en los tipos de tarjeta con mayor tasa de fraude detectada.  

---

## 🏪 3. Tipo de comercio × tipo de tarjeta (`merchant_category × card_type`)
![Mapa de calor comercio × tarjeta](../DATA/DATA_OUTPUT/EDA/operaciones_tipo_tarjeta_comercio.png)





El gráfico muestra el **reparto porcentual de las operaciones por tipo de tarjeta dentro de cada categoría de comercio**, permitiendo analizar la **preferencia de uso de las distintas marcas** (AMEX, DISCOVER, MASTERCARD y VISA) según el sector.



#### 📈 Insights:
1. **Predominio de VISA**  
   - VISA concentra la **mayor proporción de operaciones** en todos los sectores analizados, superando aproximadamente el **45–50% del total de transacciones**.  
   - Esto la posiciona como la **tarjeta más utilizada por los clientes**, reflejando su amplia aceptación y cobertura internacional.

2. **MASTERCARD con fuerte presencia**  
   - Representa cerca de un **30–35% de las operaciones**, manteniendo un peso significativo en todos los tipos de comercio.  
   - Su cuota estable sugiere **una alta penetración en comercios generalistas y minoristas**.

3. **AMEX y DISCOVER con uso limitado**  
   - Ambas presentan participaciones menores, en torno al **8–10% cada una**, con ligeras variaciones por sector.  
   - El uso de **AMEX** tiende a ser más elevado en **moda, viajes y restauración**, sectores asociados a **gasto medio-alto y clientes internacionales**.  
   - **DISCOVER**, en cambio, mantiene una presencia más homogénea pero reducida, concentrada en **pagos online o de nicho**.

#### 🧩 Conclusiones
- El mercado muestra una **clara concentración de operaciones en las redes VISA y MASTERCARD**, lo que puede implicar una **mayor exposición agregada al riesgo** en estos emisores.  
- Los comercios de **viajes, restauración y moda** son los que presentan **mayor diversidad de marcas**, posiblemente por su **perfil de cliente internacional**.  
- Esta distribución es útil para **evaluar la exposición al fraude por tipo de producto y sector**, especialmente al cruzarla con la **tasa de fraude por tarjeta** obtenida en el análisis anterior.  

---

## 🧩 4. Modo de entrada × nivel de riesgo (`entry_mode × risk_level`)
![Distribución por entry_mode y risk_level](../DATA/DATA_OUTPUT/EDA/entrymode_risklevel.png)





El mapa de calor muestra la **distribución porcentual de los niveles de riesgo (Alto, Medio, Bajo)** según el **modo de entrada de la transacción (`entry_mode`)**, ofreciendo una visión clara de qué canales presentan mayor concentración de riesgo.



#### 📈 Insights:
1. **Predominio del nivel de riesgo medio (≈40–42%)**  
   - En todos los modos de entrada, **el riesgo medio es el más frecuente**, lo que sugiere una **exposición moderada generalizada** en las operaciones.  
   - Este patrón indica que, aunque no hay un riesgo extremo predominante, **los canales mantienen una vulnerabilidad constante**.

2. **`magstripe` muestra el mayor riesgo relativo (42% medio)**  
   - Las transacciones con **banda magnética** concentran el porcentaje más alto de operaciones en nivel de riesgo medio.  
   - Esto confirma que **los métodos más antiguos presentan mayor vulnerabilidad** frente a fraude o anomalías.

3. **`ecommerce` y `wallet` mantienen un perfil equilibrado**  
   - Ambos presentan distribuciones muy próximas al promedio (≈41% medio, 31% alto, 27–28% bajo).  
   - Muestran un **riesgo moderado y estable**, propio de canales digitales que aplican validaciones adicionales.

4. **`chip` y `contactless` exhiben perfiles de riesgo controlados**  
   - Con una proporción similar de riesgo alto (≈31–32%) y bajo (≈27–28%).  
   - Esto respalda la **efectividad de las tecnologías EMV y sin contacto** en la reducción del riesgo operativo.

#### 🧩 Conclusiones
- Existe una **consistencia en la distribución de niveles de riesgo entre canales**, con predominio del **riesgo medio**, lo que sugiere una exposición homogénea.  
- **Magstripe** sigue siendo el **modo de entrada más sensible**, lo que justifica **acciones preventivas específicas** o **migración tecnológica** hacia métodos más seguros.  
- Este análisis permite **priorizar controles antifraude por canal**, alineando los esfuerzos de mitigación con la exposición al riesgo observada.


---

## 💡 5. Porcentaje de fraude por resultado (`transaction_result`)
![Fraude por resultado](../DATA/DATA_OUTPUT/EDA/fraude_por_resultado_transaccion.png)





El gráfico representa el **porcentaje de operaciones fraudulentas** según el **resultado final de la transacción** (aprobada, pendiente o declinada).  
Permite evaluar la **efectividad de los filtros de autorización** y entender en qué fase se concentran las tentativas de fraude.



#### 📈 Insights:
1. **Transacciones declinadas → mayor tasa de fraude (≈2.07%)**  
   - La mayoría de las operaciones fraudulentas se concentran en las **transacciones rechazadas**.  
   - Esto indica que **los controles antifraude están actuando correctamente**, detectando comportamientos sospechosos antes de que la operación se autorice.  

2. **Transacciones pendientes → riesgo intermedio (≈1.28%)**  
   - Los casos en espera representan una fracción relevante de operaciones potencialmente fraudulentas.  
   - Es un grupo a **monitorizar con especial atención**, ya que podrían incluir transacciones legítimas bajo revisión o intentos de fraude no confirmados.

3. **Transacciones aprobadas → menor tasa de fraude (≈1.01%)**  
   - El porcentaje más bajo se encuentra entre las **operaciones autorizadas**, lo que evidencia una **eficiente capa de prevención y filtrado previo**.  
   - Sin embargo, sigue existiendo una fracción de fraude residual que **consigue superar los filtros iniciales**.

#### 🧩 Conclusiones
- El análisis demuestra una **buena eficacia de los mecanismos de detección temprana**, que bloquean la mayoría de los intentos antes de su aprobación.  
- No obstante, el **1% de fraude aprobado** evidencia la necesidad de **reforzar controles en tiempo real y post-autorización**.  
- En conjunto, este comportamiento refleja una **operativa madura de prevención**, pero con **margen de mejora en los casos limítrofes** (pendientes o borderline).

---

## 📊 6. Evolución temporal de transacciones (mensual)
![Volumen mensual](../DATA/DATA_OUTPUT/EDA/distribucion_fraudes_por_mes.png)





El gráfico muestra la **evolución mensual del número de operaciones fraudulentas**, permitiendo detectar **picos estacionales** y posibles patrones de comportamiento a lo largo del año.



#### 📈 Insights:
1. **Enero y mayo presentan los picos más altos de fraude (46 y 45 casos respectivamente)**  
   - Enero podría reflejar un **efecto poscampaña navideña**, con operaciones tardías o devoluciones fraudulentas.  
   - Mayo puede estar asociado a **aumentos de gasto online o viajes prevacacionales**, donde se intensifican los intentos de fraude.

2. **Meses con menor actividad: febrero y noviembre (27 y 26 casos)**  
   - En estos periodos se observa una **disminución notable de intentos fraudulentos**, posiblemente por **menor volumen de transacciones** o **refuerzo de controles previos a picos estacionales**.

3. **Tendencia general estable con leves fluctuaciones**  
   - Aunque no hay una tendencia ascendente o descendente clara, sí se evidencian **picos puntuales de fraude** cada 3–4 meses, lo que sugiere **patrones cíclicos o ligados a campañas comerciales**.

#### 🧩 Conclusiones
- El fraude presenta **comportamiento estacional**, concentrándose en **momentos de alta actividad comercial**.  
- Los **meses con repunte** (enero, mayo, septiembre) deberían recibir **mayor atención en los modelos de detección** y en las estrategias preventivas.  
- Este análisis temporal permite **ajustar los recursos de monitoreo y control antifraude** de forma más eficiente a lo largo del año.

---

## 🕒 7. Evolución temporal de transacciones (día)
![Tasa de fraude mensual](../DATA/DATA_OUTPUT/EDA/Operaciones_fraudulentas_dia.png)




El gráfico presenta la **distribución diaria de las operaciones fraudulentas dentro de cada mes**, lo que permite analizar la **variabilidad del fraude a lo largo del ciclo mensual** y detectar posibles patrones de comportamiento.



#### 📈 Insights:
1. **Mayor concentración entre los días 1–5 y 24–27 del mes**  
   - Los **inicios y finales de mes** muestran un aumento notable de operaciones fraudulentas (picos de 19 a 21 casos).  
   - Este patrón puede relacionarse con **cierres de facturación, ciclos de nómina o movimientos financieros habituales**, momentos donde los defraudadores buscan camuflar sus operaciones entre transacciones legítimas.

2. **Descensos pronunciados hacia los días 10–12 y 18–22**  
   - En la parte media del mes se observan varios descensos (con mínimos de 3 y 8 casos).  
   - Esto puede indicar **periodos de menor actividad económica** o una **mejor eficacia de los sistemas de control** en esos intervalos.

3. **Actividad irregular pero con repeticiones cíclicas**  
   - Se aprecia un **ritmo de repunte cada 8–10 días**, lo que podría reflejar **patrones de prueba o “testing” de fraudes recurrentes**.

#### 🧩 Conclusiones
- El fraude presenta una **dinámica cíclica dentro del mes**, con **mayor intensidad en los extremos del periodo**.  
- Este comportamiento sugiere que los atacantes **aprovechan los momentos de mayor volumen transaccional** (cobros, pagos, cierres contables).  
- Se recomienda **reforzar los controles automáticos y alertas preventivas** durante los **primeros y últimos cinco días del mes**, cuando la exposición al riesgo es más elevada.


---

## 🕒 8. Evolución temporal de transacciones (hora/día)
![Tasa de fraude mensual](../DATA/DATA_OUTPUT/EDA/Operaciones_fraudulentas_hora_dia.png)





El gráfico combina la **cantidad total de operaciones por hora del día** (barras verdes) con la **tasa de fraude (%)** asociada (línea roja).  
Este enfoque permite identificar **horas críticas del día** donde el volumen operativo y el fraude no siguen el mismo patrón.



#### 📈 Insights:
1. **Mayor tasa de fraude entre las 2:00 y las 3:00 h (≈3%)**  
   - A pesar del bajo volumen de operaciones, el porcentaje de fraude es muy alto.  
   - Este comportamiento sugiere **actividad fraudulenta concentrada en horas de baja supervisión**, posiblemente aprovechando la menor carga operativa de control.

2. **Segundo pico relevante hacia las 15:00 h (≈3%)**  
   - Coincide con un horario de **alta actividad comercial**, lo que indica que los intentos de fraude **aumentan durante periodos de mayor flujo de transacciones**.

3. **Franja de menor riesgo: entre las 10:00 y las 13:00 h**  
   - Durante las horas laborales estándar, tanto el volumen como la tasa de fraude descienden.  
   - Esto puede asociarse a **mejoras en los controles activos** o **mayor atención de los equipos de revisión manual**.

4. **Alta actividad de operaciones entre 8:00 y 20:00 h**, pero sin correlación directa con el fraude.  
   - La mayor parte del volumen se concentra en horas diurnas, aunque la tasa de fraude **permanece moderada**.  
   - Indica que **el riesgo no depende solo del volumen**, sino de **comportamientos específicos en franjas concretas**.

#### 🧩 Conclusiones
- El fraude muestra **dos patrones horarios bien diferenciados**:  
  - **Nocturno (2–3 h)**: fraude oportunista en baja actividad.  
  - **Vespertino (15 h)**: fraude en picos de alto tráfico comercial.  
- Se recomienda **reforzar los sistemas de detección en tiempo real** durante estas franjas críticas, aplicando controles dinámicos por hora.  
- El análisis demuestra que **la vigilancia continua no debe ser homogénea**, sino **adaptativa según el riesgo horario**.


---

## 🕒 9. Tasa de fraude por país
![Tasa de fraude mensual](../DATA/DATA_OUTPUT/EDA/top_paises_tasa_fraude.png)





El gráfico muestra el **porcentaje de operaciones fraudulentas por país**, considerando los **10 países con mayor volumen de transacciones**.  
El objetivo es identificar **regiones con mayor propensión al fraude**, así como evaluar si existe **correlación entre el volumen transaccional y la incidencia delictiva**.


#### 📈 Insights:
1. **Mayores tasas de fraude en:**
   - **Netherlands Antilles (1.9%)**, **Spain (1.95%)** y **China (1.75%)**.  
     → Estos países presentan **niveles de fraude muy por encima del promedio** del grupo.  
     → En el caso de España, la cifra elevada podría estar relacionada con **el alto volumen operativo local y la diversidad de canales de pago**.  
     → En China y Antillas Neerlandesas, podría influir la **exposición a transacciones internacionales y comercio electrónico**.

2. **Riesgo medio en:**
   - **Canada (1.2%)**, **Switzerland (1.15%)**, **Eritrea (1.28%)** y **Qatar (0.8%)**.  
     → Estos países mantienen una **actividad moderada pero estable**, lo que indica **riesgo controlado pero persistente**.

3. **Menor exposición en:**
   - **Korea (0.5%)**, **Tonga (0.65%)** y **Jordan (0.95%)**.  
     → Estos mercados reflejan **baja incidencia de fraude**, posiblemente por **marcos regulatorios más estrictos o menor presencia transaccional internacional**.

#### 🧩 Conclusiones
- El fraude **no se distribuye de forma uniforme entre los países**: algunos mercados presentan **tasas elevadas sin necesariamente tener más volumen**, lo que evidencia **factores específicos de vulnerabilidad**.  
- Se recomienda **analizar la naturaleza de las operaciones internacionales** en países de alto riesgo (p. ej., **España, China y Antillas Neerlandesas**) para identificar **posibles patrones de fraude transfronterizo**.  
- Este análisis geográfico es clave para **priorizar controles adaptados al contexto regional**, reforzando la detección temprana en los países con **mayor exposición o comportamiento atípico**.

